<a href="https://colab.research.google.com/github/Juliana001/Desafios_bootcamp_luizalabs/blob/main/desafioAPI/desafioAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn python-jose passlib[bcrypt] pyngrok nest-asyncio

In [ ]:
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer
from pydantic import BaseModel
from enum import Enum
from datetime import datetime, timedelta
from jose import jwt, JWTError
from passlib.context import CryptContext
import nest_asyncio
import uvicorn


In [ ]:
SECRET_KEY = "colab-secret-key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

pwd_context = CryptContext(schemes=["bcrypt"])
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")


In [ ]:
class TransactionType(str, Enum):
    deposit = "deposit"
    withdraw = "withdraw"

class TransactionCreate(BaseModel):
    type: TransactionType
    amount: float


In [ ]:
# senha: 123456
users_db = {
    "alice": {
        "password": pwd_context.hash("123456"),
        "balance": 1000.0,
        "transactions": []
    }
}


In [ ]:
def create_access_token(data: dict):
    to_encode = data.copy()
    expire = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

def verify_token(token: str = Depends(oauth2_scheme)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload.get("sub")
    except JWTError:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token inválido"
        )


In [ ]:
app = FastAPI(title="Bank API - Colab")


In [ ]:
@app.post("/login")
async def login(username: str, password: str):
    user = users_db.get(username)

    if not user or not pwd_context.verify(password, user["password"]):
        raise HTTPException(status_code=401, detail="Credenciais inválidas")

    token = create_access_token({"sub": username})
    return {"access_token": token, "token_type": "bearer"}


In [ ]:
@app.post("/account/transaction")
async def create_transaction(
    transaction: TransactionCreate,
    username: str = Depends(verify_token)
):
    user = users_db[username]

    if transaction.type == "withdraw" and user["balance"] < transaction.amount:
        raise HTTPException(status_code=400, detail="Saldo insuficiente")

    if transaction.type == "deposit":
        user["balance"] += transaction.amount
    else:
        user["balance"] -= transaction.amount

    user["transactions"].append({
        "type": transaction.type,
        "amount": transaction.amount,
        "timestamp": datetime.utcnow()
    })

    return {"balance": user["balance"]}


In [ ]:
@app.get("/account/statement")
async def get_statement(username: str = Depends(verify_token)):
    user = users_db[username]
    return {
        "balance": user["balance"],
        "transactions": user["transactions"]
    }


In [ ]:
from pyngrok import ngrok

nest_asyncio.apply()

public_url = ngrok.connect(8000)
print("API pública:", public_url)

uvicorn.run(app, host="0.0.0.0", port=8000)
